## Beyond Bags of Features: Spatial Pyramid Matching for Recognizing Natural Scene Categories

In [0]:

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 64 Dec 14 05:53 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 4.7MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=0a79964c360d6b308b627cd5b69830b28cbe3cb524c932ff8f2f9643498e0397
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [0]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v

Kaggle API 1.5.6


In [0]:
! kaggle competitions download -c 2019-ml-finalproject
! unzip 2019-ml-finalproject.zip

In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16
! yes | pip3 install kmc2

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-4.1.2.30
     |████████████████████████████████| 25.0MB 1.2MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 1.4MB/s 
     |████████████████████████████████| 102kB 6.5MB/s 
     |████████████████████████████████| 163kB 29.9MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252243 sha256=b7a2df1fbd7a39b56ba1f8d5674c1f8d85cd3dd11d246390353782d4dc15463b
  Stored in directory: /root/.cache/p

## 라이브러리

In [0]:
import cv2
import os
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import classification_report
import scipy.cluster.vq as vq
import pandas as pd
import kmc2
from sklearn.cluster import MiniBatchKMeans


## 데이터 로드

In [0]:
df_data=pd.read_csv('./Label2Names.csv',header=None)

DATA_ROOT_TRAIN="./train"
train_des=list()
train_labels=list()

for cls in tqdm(os.listdir(DATA_ROOT_TRAIN)):
  img_list=os.listdir(DATA_ROOT_TRAIN+'/'+cls)
  img_list.sort()
  

  if cls=='BACKGROUND_Google':
    label=102
  else:
    label=(df_data.index[df_data[1]==cls]+1).tolist()[0]

  for img in img_list:
    image=cv2.imread(DATA_ROOT_TRAIN+'/'+cls+'/'+img)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=cv2.resize(image,(256,256))
    train_des.append(image)
    train_labels.append(label)

100%|██████████| 102/102 [00:03<00:00, 31.97it/s]


In [0]:
train_des=np.array(train_des)
train_labels=np.array(train_labels)

In [0]:
DATA_ROOT_TEST="./testAll_v2"
test_des=list()
img_list=os.listdir(DATA_ROOT_TEST)
img_list.sort()

for img in tqdm(img_list):
  image=cv2.imread(DATA_ROOT_TEST+'/'+img)
  image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  image=cv2.resize(image,(256,256))
  test_des.append(image)

100%|██████████| 1692/1692 [00:01<00:00, 1036.77it/s]


In [0]:
DSIFT_STEP_SIZE=8
def extract_DenseSift_descriptors(img):
  sift=cv2.xfeatures2d.SIFT_create()
  dsift_step_size=DSIFT_STEP_SIZE
  keypoints=[cv2.KeyPoint(x,y,dsift_step_size)
    for y in range(0,img.shape[0],dsift_step_size)
      for x in range(0,img.shape[1],dsift_step_size)]
  keypoints, descriptors=sift.compute(img,keypoints)
  return descriptors

In [0]:

def input_vector_encoder(feature,codebook):
  code,_=vq.vq(feature,codebook)
  word_hist,bin_edges=np.histogram(code,bins=range(codebook.shape[0]+1),normed=True)
  return word_hist

## 1. Dense sift기술자 추출

In [0]:
from time import time

t0=time()

xtrain=[]
for img in train_des:
  x=extract_DenseSift_descriptors(img)
  xtrain.append(x)

x_train_dex=np.vstack((descriptor for descriptor in xtrain))

xtest=[]
for img in test_des:
  x=extract_DenseSift_descriptors(img)
  xtest.append(x)

x_test_dex=np.vstack((descriptor for descriptor in xtest))

print(time()-t0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


359.3074679374695


## 2. Kmeans군집화로 코드북 생성

In [0]:
t0=time()

codebooksize=200
seeding=kmc2.kmc2(np.array(x_train_dex).reshape(-1,128),codebooksize)
Kmeans=MiniBatchKMeans(codebooksize,init=seeding).fit(np.array(x_train_dex).reshape(-1,128))
codebook=Kmeans.cluster_centers_

print(time()-t0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  """
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)


408.7943093776703


## 3. 기술자와 코드북을 이용해서 이미지당 히스토그램 생성

In [0]:
t0=time()

x_train=[input_vector_encoder(xtrain[i],codebook) for i in range(len(train_des))]
x_test=[input_vector_encoder(xtest[i],codebook) for i in range(len(test_des))] 

print(time()-t0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  after removing the cwd from sys.path.


18.276092052459717


In [0]:
x_train=np.asarray(x_train)
x_test=np.asarray(x_test)

## 4. 학습데이터에 대해 SVM분류기로 학습

In [0]:
t0=time()

from sklearn.svm import SVC

C_range=10.0**np.arange(-3,3)
gamma_range=10.0**np.arange(-3,3)
param_grid=dict(gamma=gamma_range.tolist(),C=C_range.tolist())

clf=GridSearchCV(SVC(),param_grid,cv=5,n_jobs=-2)
clf.fit(x_train,train_labels)

time()-t0

1376.5156416893005

## 테스트 데이터에 대한 라벨 값 예측

In [0]:
label=clf.predict(x_test)


## 제출형식

In [0]:
result=np.array(label).reshape(-1,1)
img_list=np.array(img_list).reshape(-1,1)

In [0]:
total_result=np.hstack([img_list,result])

In [0]:
df=pd.DataFrame(total_result,columns=["Id","Category"])
df.to_csv('results-hrkim-v3.csv',index=False,header=True)

In [0]:
pd.read_csv('results-hrkim-v3.csv')

,Id,Category
0,image_0001.jpg,54
1,image_0002.jpg,89
2,image_0003.jpg,92
3,image_0004.jpg,34
4,image_0005.jpg,14
...,...,...
1687,image_1688.jpg,17
1688,image_1689.jpg,61
1689,image_1690.jpg,29
1690,image_1691.jpg,38


In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-hrkim-v3.csv -m "Final_Term_Project"


100% 29.6k/29.6k [00:04<00:00, 6.66kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 

+ 성능: 0.37647~0.38238